In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols, logit
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
np.random.seed(0)


In [ ]:
Genes = pd.read_csv("ClusterDataset.csv")


In [3]:
Genes = Genes[Genes["Name"]!="gene_variance"]

In [4]:
Subjects = pd.read_csv("SubjectsMeta.txt",sep='\t')
Subjects.head(1)

,SUBJID,SEX,AGE,DTHHRDY
0,GTEX-1117F,2,60-69,4.0


In [5]:
#get all genecolumns 
gene_cols = [g for g in Genes.columns if g.startswith("ENSG")]
#Remove gene column and features column convert to numpy array
gene_array = Genes.iloc[:,1:-1].to_numpy()
norm_genes = StandardScaler().fit_transform(gene_array)
norm_genes = pd.DataFrame(norm_genes,columns=gene_cols)
norm_genes["Name"]=Genes["Name"]
norm_genes["tissue"]=Genes["tissue"]

### Merge Genes with Subject Meta Data

Leave age as a categorical variable for correction in regression

In [6]:
#use regex to get subjid take GTEX followed by sep char then match 1 or more char that are not that sep
pattern = r'^(GTEX\x2D[^\x2D]+)'
norm_genes["SUBJID"] = norm_genes["Name"].apply(lambda x: re.match(pattern,x))
#clean bad matches (need to drop variance row from preprocessing - failed to do this earlier)
norm_genes=norm_genes[~(norm_genes.index==17382)]
norm_genes["SUBJID"] = norm_genes["SUBJID"].apply(lambda x: x.group(1))

In [7]:
GeneSubj = pd.merge(norm_genes,Subjects[["SUBJID","SEX","AGE"]],on="SUBJID")
GeneSubj.head(2)

,ENSG00000244734.3,ENSG00000210082.2,ENSG00000198804.2,ENSG00000198712.1,ENSG00000198938.2,ENSG00000188536.12,ENSG00000198899.2,ENSG00000198886.2,ENSG00000275896.5,ENSG00000163220.10,...,ENSG00000148053.15,ENSG00000134291.11,ENSG00000183578.6,ENSG00000164091.11,ENSG00000173418.11,Name,tissue,SUBJID,SEX,AGE
0,-0.189282,-0.638404,-0.912095,-1.145540,-0.795922,-0.192404,-1.089657,-1.153692,-0.132456,-0.248378,...,0.062134,-0.841136,0.994222,0.596135,0.676412,GTEX-1117F-0226-SM-5GZZ7,Adipose Tissue,GTEX-1117F,2,60-69
1,-0.192726,-0.486328,0.055921,0.109423,1.286259,-0.194876,0.890270,-0.001923,-0.129289,-0.248983,...,-0.617946,-1.181526,-0.709254,-0.593598,1.388128,GTEX-1117F-0426-SM-5EGHI,Muscle,GTEX-1117F,2,60-69


### Subset to Blood and Brain and get gene_cols

In [8]:
Gene_BB = GeneSubj[GeneSubj["tissue"].isin(["Blood","Brain"])].copy()

#Make tissue, and and age categorical variables for the model (could also just do C(tissue) in the model, but this will slightly speed things up across 5k genes)
Gene_BB["tissue"] = Gene_BB["tissue"].astype("category")
Gene_BB["SEX"] = Gene_BB["SEX"].astype("category")
Gene_BB["AGE"] = Gene_BB["AGE"].astype("category")



In [9]:
Gene_BB.head(1)

,ENSG00000244734.3,ENSG00000210082.2,ENSG00000198804.2,ENSG00000198712.1,ENSG00000198938.2,ENSG00000188536.12,ENSG00000198899.2,ENSG00000198886.2,ENSG00000275896.5,ENSG00000163220.10,...,ENSG00000148053.15,ENSG00000134291.11,ENSG00000183578.6,ENSG00000164091.11,ENSG00000173418.11,Name,tissue,SUBJID,SEX,AGE
11,-0.191377,2.170896,1.860875,1.122935,1.40874,-0.191746,1.151573,1.217473,-0.132327,-0.256958,...,1.150284,-0.142,-0.414157,-1.288017,-0.963595,GTEX-1117F-3226-SM-5N9CT,Brain,GTEX-1117F,2,60-69


## Linear Modeling

In [10]:
info=[]
for gene in tqdm(gene_cols):
    #make linear model and correct for sex and age need to use Q around gene because the names are weird
    model = ols(f"Q('{gene}') ~ tissue + SEX + AGE", data=Gene_BB).fit()
    info.append({"Gene": gene,"coef": model.params["tissue[T.Brain]"],"pval": model.pvalues["tissue[T.Brain]"]})
tissue_eval = pd.DataFrame(info)

100%|██████████| 5000/5000 [00:49<00:00, 100.44it/s]


In [11]:
#read in gene descriptions
get_description = pd.read_csv("Gene_Descriptions.csv")
get_description = dict(zip(get_description["Name"],get_description["Desc"]))

In [13]:
tissue_eval["Desc"] = tissue_eval["Gene"].map(get_description)

#### Top 5 most signficant pvalues

In [14]:
tissue_eval.sort_values("pval").head(5)

,Gene,coef,pval,Desc
1535,ENSG00000064601.16,-1.183147,0.0,CTSA
3162,ENSG00000105357.15,0.430296,0.0,MYH14
3155,ENSG00000002834.17,-1.666254,0.0,LASP1
3154,ENSG00000069275.12,1.681651,0.0,NUCKS1
3146,ENSG00000153113.23,-0.587200,0.0,CAST


In [15]:
print("Number of underflow pvalues:",tissue_eval[tissue_eval["pval"]==0]["Gene"].count())
print("Number of not signficant genes:",tissue_eval[tissue_eval["pval"]>.05]["Gene"].count())

Number of underflow pvalues: 967
Number of not signficant genes: 294


#### Top 5 most highest effect genes

In [16]:
tissue_eval["abs_coef"]=tissue_eval["coef"].abs()
tissue_eval.sort_values("abs_coef",ascending=False).head(5)

,Gene,coef,pval,Desc,abs_coef
638,ENSG00000128340.14,-3.965246,0.0,RAC2,3.965246
493,ENSG00000136167.13,-3.914198,0.0,LCP1,3.914198
1255,ENSG00000147168.12,-3.862512,0.0,IL2RG,3.862512
1777,ENSG00000130592.15,-3.860039,0.0,LSP1,3.860039
449,ENSG00000162511.7,-3.834085,0.0,LAPTM5,3.834085


Interpretation:

There is not overlap between the the most significant genes and the highest effect (coef). Two reasons:

1) The pvalues are extremely tiny and for both sets and underflow to 0 (would be reported as <0.0001), so sorting a bunch of 0s is useless. There are much greater than only 5 values that underflow to 0 (967 genes) making top 5 pvalues meaningless. 

2) Pvalue and abs coef are different. Pvalue controls for both noise and sample size to see the strength of dif between brain and blood for that gene. The coef simply how big the gene difference is between brain and blood ignoring noise and sample size.

Since Brain and Blood are extremely different tissues it is not surprising to see many very sig dif genes. In fact, only 295 genes of 5000 are NOT significantly different. Thus, it is unsurprising that there is not overlap in just taking the top 5 smallest pvalues because it is essentially a random sample of 5 of the 967 most significant genes (where the pval underflows to 0)

## Logistic Modeling

Predict if gene expression can predict sex in blood only

In [17]:
Gene_Blood = Gene_BB[Gene_BB["tissue"]=="Blood"].copy()
#make sex a binary numerical variable 0,1
Gene_Blood["SEX"] = (Gene_Blood["SEX"] == 2).astype("int")

In [18]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

info_log=[]
for gene in tqdm(gene_cols):
    log_model = logit(f"SEX ~ Q('{gene}')", data=Gene_Blood).fit(disp=0)
    info_log.append({"Gene": gene,"coef": log_model.params.iloc[1],"pval": log_model.pvalues.iloc[1]})
    
log_SEX_eval = pd.DataFrame(info_log)

100%|██████████| 5000/5000 [00:24<00:00, 205.76it/s]


In [19]:
log_SEX_eval["Desc"] = log_SEX_eval["Gene"].map(get_description)

#### Top 5 most signficant pvalues logistic

In [20]:
log_SEX_eval.sort_values("pval").head(5)

,Gene,coef,pval,Desc
3386,ENSG00000229807.10,482.625332,4.069779e-20,XIST
349,ENSG00000198034.10,0.425305,6.965764e-12,RPS4X
2638,ENSG00000215301.9,0.491742,4.678563e-11,DDX3X
4199,ENSG00000067048.16,-187.174051,1.517101e-09,DDX3Y
3021,ENSG00000130985.16,0.379586,1.583458e-08,UBA1


In [21]:
print("Number of underflow pvalues:",log_SEX_eval[log_SEX_eval["pval"]==0]["Gene"].count())
print("Number of not signficant genes:",log_SEX_eval[log_SEX_eval["pval"]<.05]["Gene"].count())
print("Number of not signficant genes:",log_SEX_eval[log_SEX_eval["pval"]>.05]["Gene"].count())

Number of underflow pvalues: 0
Number of not signficant genes: 541
Number of not signficant genes: 4459


Notes: Most significant gene is XIST which makes complete sense because this gene silences one X chromosome in females. This gene would not be expressed in males. The next three genes have related expression to the X and Y chromosomes (X or Y after the gene show this and confirmed in gene cards) DDX3Y is strictly Y linked, so this also makes sense.

#### Top 5 most highest effect genes

In [22]:
log_SEX_eval["abs_coef"]=log_SEX_eval["coef"].abs()
log_SEX_eval.sort_values("abs_coef",ascending=False).head(5)

,Gene,coef,pval,Desc,abs_coef
4751,ENSG00000012817.15,-3962.612072,0.994726,KDM5D,3962.612072
1159,ENSG00000129824.15,-1422.133879,0.999319,RPS4Y1,1422.133879
723,ENSG00000135222.6,1307.963203,0.119342,CSN2,1307.963203
770,ENSG00000124157.6,-1034.619007,0.120261,SEMG2,1034.619007
406,ENSG00000171209.3,925.322653,0.092940,CSN3,925.322653


Interpretation:

To revisit what I stated in the prior section about how pvalue and coef are different: pvalue controls for both noise and sample size to see the strength of dif for the gene. The coef simply how big the gene difference is between male and female ignoring how noisy it may be. This is likely why none of the biggest coefs are significant. 